In [1]:
import os
import argparse
import math
import gym
import numpy as np
import itertools
import torch
from PIL import Image
from SAC.sac import SAC
from tensorboardX import SummaryWriter
from SAC.normalized_actions import NormalizedActions
from SAC.replay_memory import ReplayMemory

'''
parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
parser.add_argument('--env-name', default="BipedalWalker-v2",
                    help='name of the environment to run')
parser.add_argument('--policy', default="Gaussian",
                    help='algorithm to use: Gaussian | Deterministic')
parser.add_argument('--eval', type=bool, default=False,
                    help='Evaluate a policy (default:False)')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor for reward (default: 0.99)')
parser.add_argument('--tau', type=float, default=0.005, metavar='G',
                    help='target smoothing coefficient(τ) (default: 0.005)')
parser.add_argument('--lr', type=float, default=0.0003, metavar='G',
                    help='learning rate (default: 0.0003)')
parser.add_argument('--alpha', type=float, default=0.2, metavar='G',
                    help='Temperature parameter α determines the relative importance of the entropy term against the reward (default: 0.2)')
parser.add_argument('--seed', type=int, default=543, metavar='N',
                    help='random seed (default: 543)')
parser.add_argument('--batch_size', type=int, default=256, metavar='N',
                    help='batch size (default: 256)')
parser.add_argument('--num_steps', type=int, default=1000000, metavar='N',
                    help='maximum number of steps (default: 1000000)')
parser.add_argument('--hidden_size', type=int, default=256, metavar='N',
                    help='hidden size (default: 256)')
parser.add_argument('--updates_per_step', type=int, default=1, metavar='N',
                    help='model updates per simulator step (default: 1)')
parser.add_argument('--target_update_interval', type=int, default=1, metavar='N',
                    help='Value target update per no. of updates per step (default: 1)')
parser.add_argument('--replay_size', type=int, default=1000000, metavar='N',
                    help='size of replay buffer (default: 10000000)')
args = parser.parse_args()
'''

args = {
    "env_name": "BipedalWalker-v2",
    "policy": "Gaussian",
    "eval": False,
    "gamma": 0.99,
    "tau": 0.001,
    "lr": 0.0001,
    "alpha": 0.2,
    "seed": 333,
    "batch_size": 1024,
    "num_steps": 5000000,
    "steps_in_episode": 2000,
    "hidden_size": 256,
    "updates_per_step": 1,
    "target_update_interval": 1,
    "replay_size": 20000
}    


# Environment
env = NormalizedActions(gym.make(args['env_name']))
env.seed(args['seed'])
torch.manual_seed(args['seed'])
np.random.seed(args['seed'])

# Agent
agent = SAC(env.observation_space.shape[0], env.action_space, args)

writer = SummaryWriter()

# Memory
memory = ReplayMemory(args['replay_size'])

# Training Loop
rewards = []
total_numsteps = 0
updates = 0

for i_episode in itertools.count():
    state = env.reset()

    episode_reward = 0
    for t in range(args['steps_in_episode']):
        action = agent.select_action(state)  # Sample action from policy
        next_state, reward, done, _ = env.step(action)  # Step
        mask = not done  # 1 for not done and 0 for done
        memory.push(state, action, reward, next_state, mask)  # Append transition to memory
        if len(memory) > args['batch_size']:
            for i in range(args['updates_per_step']): # Number of updates per step in environment
                # Sample a batch from memory
                state_batch, action_batch, reward_batch, next_state_batch, mask_batch = memory.sample(args['batch_size'])
                # Update parameters of all the networks
                value_loss, critic_1_loss, critic_2_loss, policy_loss = agent.update_parameters(state_batch, action_batch, 
                                                                                                reward_batch, next_state_batch, 
                                                                                                mask_batch, updates)

                writer.add_scalar('loss/value', value_loss, updates)
                writer.add_scalar('loss/critic_1', critic_1_loss, updates)
                writer.add_scalar('loss/critic_2', critic_2_loss, updates)
                writer.add_scalar('loss/policy', policy_loss, updates)
                updates += 1

        state = next_state
        total_numsteps += 1
        episode_reward += reward

        if done:
            break

    if total_numsteps > args['num_steps']:
        break

    writer.add_scalar('reward/train', episode_reward, i_episode)
    rewards.append(episode_reward)
    print("Episode: {}, total numsteps: {}, reward: {}, average reward: {}".format(i_episode, total_numsteps, np.round(rewards[-1],2),
                                                                                np.round(np.mean(rewards[-100:]),2)))




WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: <class 'SAC.normalized_actions.NormalizedActions'> doesn't implement 'action' method. Maybe it implements deprecated '_action' method.
Episode: 0, total numsteps: 74, reward: -105.35, average reward: -105.35
Episode: 1, total numsteps: 147, reward: -103.44, average reward: -104.4
Episode: 2, total numsteps: 1747, reward: -109.67, average reward: -106.15
Episode: 3, total numsteps: 2408, reward: -173.73, average reward: -123.05
Episode: 4, total numsteps: 2449, reward: -109.1, average reward: -120.26
Episode: 5, total numsteps: 4049, reward: -82.46, average reward: -113.96
Episode: 6, total numsteps: 4118, reward: -112.13, average reward: -113.7
Episode: 7, total numsteps: 4187, reward: -107.05, average reward: -112.87
Episode: 8, total numsteps: 4248, reward: -114.06, average reward: -

Episode: 104, total numsteps: 75067, reward: -78.76, average reward: -100.15
Episode: 105, total numsteps: 76667, reward: -74.73, average reward: -100.07
Episode: 106, total numsteps: 78267, reward: -74.85, average reward: -99.7
Episode: 107, total numsteps: 79867, reward: -88.35, average reward: -99.51
Episode: 108, total numsteps: 81467, reward: -72.04, average reward: -99.09
Episode: 109, total numsteps: 83067, reward: -75.55, average reward: -98.7
Episode: 110, total numsteps: 84667, reward: -77.9, average reward: -98.33
Episode: 111, total numsteps: 86267, reward: -74.13, average reward: -97.82
Episode: 112, total numsteps: 87867, reward: -78.64, average reward: -97.37
Episode: 113, total numsteps: 89467, reward: -81.57, average reward: -97.04
Episode: 114, total numsteps: 91067, reward: -75.53, average reward: -96.59
Episode: 115, total numsteps: 92667, reward: -76.13, average reward: -96.24
Episode: 116, total numsteps: 94267, reward: -72.74, average reward: -95.81
Episode: 117,

Episode: 211, total numsteps: 246267, reward: -65.06, average reward: -74.21
Episode: 212, total numsteps: 247867, reward: -70.44, average reward: -74.13
Episode: 213, total numsteps: 249467, reward: -60.37, average reward: -73.92
Episode: 214, total numsteps: 251067, reward: -64.81, average reward: -73.81
Episode: 215, total numsteps: 252667, reward: -64.27, average reward: -73.69
Episode: 216, total numsteps: 254267, reward: -68.22, average reward: -73.65
Episode: 217, total numsteps: 255867, reward: -64.93, average reward: -73.52
Episode: 218, total numsteps: 257467, reward: -60.89, average reward: -73.39
Episode: 219, total numsteps: 259067, reward: -60.68, average reward: -73.23
Episode: 220, total numsteps: 260667, reward: -64.17, average reward: -73.07
Episode: 221, total numsteps: 262267, reward: -67.99, average reward: -72.96
Episode: 222, total numsteps: 263867, reward: -68.6, average reward: -72.85
Episode: 223, total numsteps: 265467, reward: -68.05, average reward: -72.82


Episode: 318, total numsteps: 417467, reward: -67.28, average reward: -67.61
Episode: 319, total numsteps: 419067, reward: -72.28, average reward: -67.72
Episode: 320, total numsteps: 420667, reward: -70.11, average reward: -67.78
Episode: 321, total numsteps: 422267, reward: -68.14, average reward: -67.78
Episode: 322, total numsteps: 423867, reward: -66.79, average reward: -67.77
Episode: 323, total numsteps: 425467, reward: -72.72, average reward: -67.81
Episode: 324, total numsteps: 427067, reward: -72.82, average reward: -67.89
Episode: 325, total numsteps: 428667, reward: -70.4, average reward: -67.91
Episode: 326, total numsteps: 430267, reward: -70.0, average reward: -67.99
Episode: 327, total numsteps: 431867, reward: -67.58, average reward: -68.01
Episode: 328, total numsteps: 433467, reward: -70.29, average reward: -68.05
Episode: 329, total numsteps: 435067, reward: -71.89, average reward: -68.1
Episode: 330, total numsteps: 436667, reward: -74.69, average reward: -68.13
Ep

Episode: 425, total numsteps: 515871, reward: -111.69, average reward: -89.53
Episode: 426, total numsteps: 515925, reward: -112.79, average reward: -89.95
Episode: 427, total numsteps: 515985, reward: -121.61, average reward: -90.49
Episode: 428, total numsteps: 516027, reward: -109.66, average reward: -90.89
Episode: 429, total numsteps: 516090, reward: -123.0, average reward: -91.4
Episode: 430, total numsteps: 516152, reward: -114.03, average reward: -91.79
Episode: 431, total numsteps: 516212, reward: -118.35, average reward: -92.25
Episode: 432, total numsteps: 516275, reward: -120.7, average reward: -92.69
Episode: 433, total numsteps: 516326, reward: -113.05, average reward: -93.04
Episode: 434, total numsteps: 516358, reward: -106.31, average reward: -93.37
Episode: 435, total numsteps: 516403, reward: -110.24, average reward: -93.23
Episode: 436, total numsteps: 516457, reward: -112.37, average reward: -93.67
Episode: 437, total numsteps: 516506, reward: -111.32, average rewa

...

Episode: 3834, total numsteps: 2409627, reward: -105.08, average reward: -102.26
Episode: 3835, total numsteps: 2409703, reward: -102.48, average reward: -102.76
Episode: 3836, total numsteps: 2409803, reward: -105.01, average reward: -103.28
Episode: 3837, total numsteps: 2409848, reward: -102.35, average reward: -103.52
Episode: 3838, total numsteps: 2409913, reward: -101.0, average reward: -103.52
Episode: 3839, total numsteps: 2410002, reward: -104.08, average reward: -103.15
Episode: 3840, total numsteps: 2410116, reward: -105.93, average reward: -103.55
Episode: 3841, total numsteps: 2410182, reward: -100.26, average reward: -103.57
Episode: 3842, total numsteps: 2410262, reward: -102.16, average reward: -103.04
Episode: 3843, total numsteps: 2410346, reward: -104.59, average reward: -102.49
Episode: 3844, total numsteps: 2410446, reward: -102.5, average reward: -102.3
Episode: 3845, total numsteps: 2410545, reward: -105.01, average reward: -102.05
Episode: 3846, total numsteps: 

Episode: 3936, total numsteps: 2419425, reward: -105.04, average reward: -104.25
Episode: 3937, total numsteps: 2419501, reward: -100.86, average reward: -104.23
Episode: 3938, total numsteps: 2419590, reward: -103.5, average reward: -104.26
Episode: 3939, total numsteps: 2419648, reward: -102.58, average reward: -104.24
Episode: 3940, total numsteps: 2419712, reward: -110.82, average reward: -104.29
Episode: 3941, total numsteps: 2419767, reward: -101.28, average reward: -104.3
Episode: 3942, total numsteps: 2419842, reward: -102.76, average reward: -104.31
Episode: 3943, total numsteps: 2419896, reward: -101.96, average reward: -104.28
Episode: 3944, total numsteps: 2419964, reward: -103.32, average reward: -104.29
Episode: 3945, total numsteps: 2420067, reward: -102.57, average reward: -104.26
Episode: 3946, total numsteps: 2420140, reward: -101.23, average reward: -104.25
Episode: 3947, total numsteps: 2420202, reward: -100.82, average reward: -104.24
Episode: 3948, total numsteps:

Episode: 4038, total numsteps: 2426691, reward: -100.32, average reward: -103.57
Episode: 4039, total numsteps: 2426750, reward: -109.55, average reward: -103.64
Episode: 4040, total numsteps: 2426848, reward: -114.88, average reward: -103.68
Episode: 4041, total numsteps: 2426917, reward: -102.66, average reward: -103.69
Episode: 4042, total numsteps: 2427020, reward: -100.71, average reward: -103.67
Episode: 4043, total numsteps: 2427103, reward: -100.53, average reward: -103.66
Episode: 4044, total numsteps: 2427201, reward: -98.7, average reward: -103.61
Episode: 4045, total numsteps: 2427302, reward: -111.32, average reward: -103.7
Episode: 4046, total numsteps: 2427351, reward: -107.06, average reward: -103.75
Episode: 4047, total numsteps: 2427506, reward: -104.56, average reward: -103.79
Episode: 4048, total numsteps: 2427568, reward: -108.57, average reward: -103.73
Episode: 4049, total numsteps: 2427638, reward: -107.45, average reward: -103.72
Episode: 4050, total numsteps: 

Episode: 4140, total numsteps: 2457676, reward: -118.5, average reward: -115.86
Episode: 4141, total numsteps: 2457792, reward: -111.26, average reward: -115.94
Episode: 4142, total numsteps: 2457892, reward: -114.51, average reward: -116.08
Episode: 4143, total numsteps: 2457970, reward: -120.81, average reward: -116.28
Episode: 4144, total numsteps: 2458099, reward: -117.04, average reward: -116.47
Episode: 4145, total numsteps: 2458235, reward: -108.23, average reward: -116.44
Episode: 4146, total numsteps: 2459835, reward: -86.02, average reward: -116.23
Episode: 4147, total numsteps: 2461435, reward: -83.81, average reward: -116.02
Episode: 4148, total numsteps: 2461551, reward: -130.11, average reward: -116.23
Episode: 4149, total numsteps: 2461713, reward: -114.33, average reward: -116.3
Episode: 4150, total numsteps: 2462037, reward: -136.38, average reward: -116.62
Episode: 4151, total numsteps: 2463637, reward: -87.07, average reward: -116.24
Episode: 4152, total numsteps: 24

KeyboardInterrupt: 

In [ ]:
def test(agent):   
    random_seed = 0
    episodes = 3
    max_timesteps = 2000
    render = True
    save_gif = True
     
    for i_episode in range(1, episodes):
        state = env.reset()
        episode_reward = 0
        for t in range(10000):  # Don't infinite loop while learning
            action = agent.select_action(state)  # Sample action from policy
            next_state, reward, done, _ = env.step(action)  # Step
           
            if render:
                env.render()  
                if save_gif:
                    dirname = './gif/sac/{}'.format(i_episode)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/sac/{}/{}.jpg'.format(i_episode,t))

            state = next_state            
            episode_reward += reward

            if done:
                break    
   
            
        print('Episode: {}\tReward: {}'.format(i_episode, int(episode_reward)))
        running_reward = 0
        env.close()        
                
test(agent)